In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

# Directorios de datos
train_dir = 'C:\\Eder\\Cosas TEC\\9 Semestre\\Inteligencia Artificial\\Dataset flores\\Train'
test_dir = 'C:\\Eder\\Cosas TEC\\9 Semestre\\Inteligencia Artificial\\Dataset flores\\Test'

# Parámetros del modelo
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
epochs = 10
batch_size = 16  # Intenta con 16 o 8, ajusta según sea necesario

# Generadores de datos con aumento para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Imprime el número de muestras de entrenamiento antes de model.fit()
print("Número de muestras de entrenamiento:", train_generator.samples)

# Asegurarse de que hay al menos una muestra de entrenamiento y que batch_size es un divisor de train_generator.samples
assert train_generator.samples > 0, "Número de muestras de entrenamiento es cero"
assert train_generator.samples % batch_size == 0, "batch_size no es un divisor exacto de train_generator.samples"

# Construir el modelo CNN
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Guardar el modelo
model.save('modelo_flores.keras')





Found 400 images belonging to 5 classes.
Found 8 images belonging to 5 classes.
Número de muestras de entrenamiento: 400


C:\Users\esque\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10


C:\Users\esque\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 214ms/step - accuracy: 0.2274 - loss: 1.5843 - val_accuracy: 0.2500 - val_loss: 1.4241
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.2500 - val_loss: 1.4241
Epoch 3/10


C:\Users\esque\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 235ms/step - accuracy: 0.5287 - loss: 1.3230 - val_accuracy: 0.6250 - val_loss: 1.1102
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6250 - val_loss: 1.1102
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 251ms/step - accuracy: 0.5458 - loss: 1.0410 - val_accuracy: 0.7500 - val_loss: 0.7993
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.7993
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 252ms/step - accuracy: 0.6562 - loss: 0.8809 - val_accuracy: 0.7500 - val_loss: 0.6883
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.7500 - val_loss: 0.6883
Epoch 9/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 251ms/step - accuracy: 0.6893 - loss: 0.7700 - val_accuracy: 0.6250 - val_loss: 0.6261
Epoch 10/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - va

C:\Users\esque\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\models\model.py:342: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  warnings.warn(
